In [5]:
import random
import threading
import time
import json
from blockchain import*

class node:
    def __init__(self, blockchain, url):
        self.url = url
        self.nodes = []
        self.blockchain = blockchain
        self.syncer = None
        self.syncer_timeout = random.uniform(1, 5)
        self.timer = threading.Timer(self.syncer_timeout, self.start_election)
        self.timer.start()

    def start_election(self):
        votes_received = 0
        for node in self.nodes:
            vote = node.request_vote(self.url, self.blockchain.getPreviousHash())
            if vote:
                votes_received += 1
        
        if votes_received >= len(self.nodes) // 2:
            self.syncer = self.url
            self.send_heartbeat()

    def request_vote(self, candidate_url, candidate_prev_hash):
        if self.blockchain.getPreviousHash() == candidate_prev_hash:
            return True
        return False

    def send_heartbeat(self):
        while self.syncer == self.url:
            time.sleep(5)
            for node in self.nodes:
                node.receive_heartbeat(self.url, self.blockchain.getPreviousHash())

    def receive_heartbeat(self, syncer_url, syncer_prev_hash):
        if self.blockchain.getPreviousHash() == syncer_prev_hash:
            self.syncer = syncer_url
            self.timer.cancel()
            self.syncer_timeout = random.uniform(1, 5)
            self.timer = threading.Timer(self.syncer_timeout, self.start_election)
            self.timer.start()

    def add_block(self, new_block):
        if self.syncer is not None:
            if self.blockchain.getPreviousHash() == self.nodes[self.syncer].blockchain.getPreviousHash():
                self.blockchain.add_new_block(new_block)
                for node in self.nodes:
                    node.receive_block(new_block)

    def receive_block(self, new_block):
        if self.blockchain.verify_new_block(new_block):
            self.blockchain.add_new_block(new_block)
